In [2]:
import os
import os.path as path
import csv
import numpy as np

In [ ]:
dataset_path = r"./dat/data-set-2016-TiO2/"

In [9]:
def check_periodics():
    count = 0

    # ripped and slightly modified from xsf_clean.py
    for s_no in range(1, 7816):
        file = path.join(
            dataset_path,
            f"structure{str(s_no).zfill(4)}.xsf"
        )
    
        with open(file, "r") as f:
            lines = f.readlines()
            if "PRIMVEC\n" in lines:
                count += 1

    print(f"non-periodic file count: {count}")

check_periodics()

non-periodic file count: 7815


In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# suppress tensorflow warnings
import tensorflow as tf
import keras

from keras.models import Sequential 
from keras import Input
from keras import layers
from keras.layers import Dense, \
                         Dropout

In [4]:
class BPNN(keras.Model):
    """
    The Model discussed in the 2007 paper by
    Behler and Parinello.

    This class contains an adaptive model which is
    the subnet for one atom.

    The subnet takes all the atom coordinates,
    and outputs the atomic contribution of energy
    """
    def __init__(self, ls: list, r_c, r_s, eta1, eta2, lbd, zeta):
        super().__init__()

        # properties of the subnet
        self._input  = Input(shape = (2, ))
        self.ls  = ls
        self._output = Dense(1, activation = "softmax")
        
        self.r_c  = r_c   # angstrom
        self.r_s  = r_s 
        self.eta1 = eta1
        self.eta2 = eta2
        self.lbd  = 1 if lbd == 1 else -1
        self.zeta = zeta

    def _norm(self, r):
        """
        macro for 2-d norm
        """
        return np.sqrt(np.dot(r, r))

    def _cut_off(self, r: float) -> float:
        """
        radial cutoff function
        """
        out = 0
        if r <= self.r_c:
            out = 0.5 * np.cos(np.pi * (r/self.r_c) + 1)
    
        return out

    def _trim_padding(self, inputs):
        mask = tf.reduce_any(inputs != -99, axis=-1)
        mask = tf.cast(mask, dtype=inputs.dtype)
               
        return inputs * tf.expand_dims(mask, axis=-1)

    def call(self, inputs, training = False):
        """
        Feed-forward algorithm for the model.
        calculate the values of the symmetry functions
        for each atom and applies to the same layers.
        """
        
        inputs = self._trim_padding(inputs)
        e_curr = 0
        g1s = self.get_g1is(inputs)
        g2s = self.get_g2is(inputs)

        for idx, in_vec in enumerate(zip(g1s, g2s)):
            in_vec = np.array(in_vec)
            x = self._input(in_vec)
            
            for layer in self.layers:
                # dropout only applies on training
                # for reqularisation
                if isinstance(layer, Dropout):
                    x = layer(x, training = training)
                    
                x = layer(x)

            e_curr += self._output(x).output

            # sum of all the atomic energy contributions
            # non trainable (just a linear sum)
            e_total = Dense(1, trainable = False)(e_curr)
        return e_total

In [5]:
import pandas as pd
from sklearn.utils import shuffle

N_SYSTEMS = 7815

# read all systems
X = np.array(
    [
        pd.read_csv(
            f"./dat/positions/pos_{str(idx).zfill(4)}.csv"
        ).to_numpy()
        for idx in range(1, N_SYSTEMS + 1)
    ]
)

# read energy file
y = pd.read_csv(
    "./dat/energies.csv"
)

In [6]:
N_atoms = np.array([np.shape(x)[0] for x in X])
np.unique(N_atoms)

array([95])

In [7]:
X, y = shuffle(X, y)

lo = int(0.6 * N_SYSTEMS)
hi = int(0.8 * N_SYSTEMS)

X_train = X[:lo]
X_val   = X[lo:hi]
X_test  = X[hi:]

y_train = y[:lo]
y_val   = y[lo:hi]
y_test  = y[hi:]

In [8]:
model = BPNN([
        Dense(3, activation = "relu"),
        Dense(3, activation = "relu")
    ],
    r_c = 6, r_s = 0,
    eta1 = 0.003214, 
    eta2 = 0.00357, 
    lbd = -1, zeta = 1)

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    loss = keras.losses.MeanSquaredError()
)

In [ ]:
bpnn = model.fit(
    X_train, y_train,
    batch_size = 300,
    epochs = 100,
    validation_data = (X_val, y_val),
    verbose = 0
)

In [1]:
from SymmetryCalculator import SymmetryCalculator

In [2]:
sym_calc = SymmetryCalculator()
sym_calc.write_symmetries(r"./dat/symmetries/", 1)